In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score,\
                                    GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor,\
                             RandomForestRegressor

In [2]:
covid = pd.read_csv('../data/nyt_county.csv')

In [3]:
print(covid.shape)
covid.head()

(1381436, 6)


,date,county,state_name,county_fips_code,confirmed_cases,deaths
0,2021-02-19,Bristol,Rhode Island,44001.0,4115,123.0
1,2021-02-19,Kent,Rhode Island,44003.0,14896,291.0
2,2021-02-19,Providence,Rhode Island,44007.0,80264,1730.0
3,2021-02-19,Chittenden,Vermont,50007.0,4590,83.0
4,2021-02-20,Apache,Arizona,4001.0,10244,365.0


In [4]:
len(covid.county_fips_code.unique())

3219

In [5]:
census = pd.read_csv('../data/2018_5yr.csv')

In [6]:
print(census.shape)
census.head()

(3220, 242)


,geo_id,do_date,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,...,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english
0,35039,2014-01-01,39307,12398,19250,20057,40.6,1224,1240,1278,...,NaN,NaN,2170,1985,1970,9260,1653,NaN,NaN,NaN
1,72133,2014-01-01,22066,7465,10650,11416,37.8,567,662,864,...,2171.0,1242.0,1155,1362,1319,5884,1410,NaN,NaN,NaN
2,72043,2014-01-01,39265,13346,19056,20209,40.7,945,1109,1431,...,3023.0,2529.0,1851,2144,1901,9475,2219,NaN,NaN,NaN
3,72151,2014-01-01,34149,11722,16541,17608,42.5,754,1095,1020,...,1969.0,1700.0,1957,1388,1933,8055,2147,NaN,NaN,NaN
4,72071,2014-01-01,42420,15012,20629,21791,41.6,898,1159,1381,...,3246.0,2725.0,1996,2057,2211,10099,2551,NaN,NaN,NaN


In [7]:
covid.groupby('county_fips_code').max().isna().sum()

date                0
county              0
state_name          0
confirmed_cases     0
deaths             78
dtype: int64

In [8]:
by_county = covid.groupby('county_fips_code').max()

In [9]:
by_county

,date,county,state_name,confirmed_cases,deaths
county_fips_code,,,,,
1001.0,2021-06-02,Autauga,Alabama,7161,111.0
1003.0,2021-06-02,Baldwin,Alabama,21674,311.0
1005.0,2021-06-02,Barbour,Alabama,2340,59.0
1007.0,2021-06-02,Bibb,Alabama,2666,64.0
1009.0,2021-06-02,Blount,Alabama,6889,139.0
...,...,...,...,...,...
72151.0,2021-06-02,Yabucoa,Puerto Rico,1392,NaN
72153.0,2021-06-02,Yauco,Puerto Rico,1255,NaN
78010.0,2021-06-02,St. Croix,Virgin Islands,1398,10.0


In [10]:
census.set_index('geo_id', inplace=True)

In [11]:
census

,do_date,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,male_15_to_17,...,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english
geo_id,,,,,,,,,,,,,,,,,,,,,
35039,2014-01-01,39307,12398,19250,20057,40.6,1224,1240,1278,846,...,NaN,NaN,2170,1985,1970,9260,1653,NaN,NaN,NaN
72133,2014-01-01,22066,7465,10650,11416,37.8,567,662,864,547,...,2171.0,1242.0,1155,1362,1319,5884,1410,NaN,NaN,NaN
72043,2014-01-01,39265,13346,19056,20209,40.7,945,1109,1431,801,...,3023.0,2529.0,1851,2144,1901,9475,2219,NaN,NaN,NaN
72151,2014-01-01,34149,11722,16541,17608,42.5,754,1095,1020,714,...,1969.0,1700.0,1957,1388,1933,8055,2147,NaN,NaN,NaN
72071,2014-01-01,42420,15012,20629,21791,41.6,898,1159,1381,877,...,3246.0,2725.0,1996,2057,2211,10099,2551,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8031,2014-01-01,693417,294358,347188,346229,34.4,22097,21155,18037,9721,...,182725.0,78341.0,32344,28425,25135,156010,35635,NaN,NaN,NaN
34017,2014-01-01,668631,255429,332268,336363,35.1,24092,18265,17642,10232,...,143998.0,73093.0,28683,26929,26028,151620,34256,NaN,NaN,NaN
24510,2014-01-01,614700,238436,288691,326009,35.1,20357,18021,16926,9570,...,119779.0,55793.0,27604,26474,26071,153745,37059,NaN,NaN,NaN


In [12]:
df = pd.merge(by_county, census, left_index = True, right_index=True)

In [13]:
print('by_county df:', by_county.shape)
print('census df:', census.shape)
print('final df:', df.shape)

by_county df: (3218, 5)
census df: (3220, 241)
final df: (3211, 246)


In [14]:
df

,date,county,state_name,confirmed_cases,deaths,do_date,total_pop,households,male_pop,female_pop,...,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english
1001,2021-06-02,Autauga,Alabama,7161,111.0,2014-01-01,55200,21115,26874,28326,...,9336.0,5331.0,3009,2823,3245,13435,1865,NaN,NaN,NaN
1003,2021-06-02,Baldwin,Alabama,21674,311.0,2014-01-01,208107,78622,101188,106919,...,33923.0,22509.0,9861,11627,10456,45076,6674,NaN,NaN,NaN
1005,2021-06-02,Barbour,Alabama,2340,59.0,2014-01-01,25782,9186,13697,12085,...,2261.0,1729.0,1106,1280,1424,5216,704,NaN,NaN,NaN
1007,2021-06-02,Bibb,Alabama,2666,64.0,2014-01-01,22527,6840,12152,10375,...,1775.0,1531.0,922,1154,1169,4424,757,NaN,NaN,NaN
1009,2021-06-02,Blount,Alabama,6889,139.0,2014-01-01,57645,20600,28434,29211,...,6385.0,4569.0,3064,2998,3180,12220,1718,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72145,2021-06-02,Vega Baja,Puerto Rico,3442,NaN,2014-01-01,53371,18081,25580,27791,...,4179.0,3476.0,2280,3023,2658,12419,2978,NaN,NaN,NaN
72147,2021-06-02,Vieques,Puerto Rico,205,NaN,2014-01-01,8771,2470,4332,4439,...,609.0,423.0,435,441,248,1497,163,NaN,NaN,NaN
72149,2021-06-02,Villalba,Puerto Rico,878,NaN,2014-01-01,22993,7712,11169,11824,...,1578.0,1388.0,1175,1102,1496,6008,1618,NaN,NaN,NaN
72151,2021-06-02,Yabucoa,Puerto Rico,1392,NaN,2014-01-01,34149,11722,16541,17608,...,1969.0,1700.0,1957,1388,1933,8055,2147,NaN,NaN,NaN


In [15]:
df.isna().sum().sort_values(ascending=False)[:20]

speak_spanish_at_home_low_english          3211
pop_15_and_over                            3211
pop_widowed                                3211
pop_separated                              3211
pop_now_married                            3211
pop_never_married                          3211
pop_5_years_over                           3211
speak_only_english_at_home                 3211
speak_spanish_at_home                      3211
pop_divorced                               3211
aggregate_travel_time_to_work               154
population_1_year_and_over                   79
different_house_year_ago_same_city           79
less_than_high_school_graduate               79
graduate_professional_degree                 79
high_school_including_ged                    79
bachelors_degree_2                           79
some_college_and_associates_degree           79
different_house_year_ago_different_city      79
deaths                                       78
dtype: int64

In [16]:
for col_name in df.columns:
    if 'poverty' in col_name:
        print(col_name)

pop_determined_poverty_status
poverty


In [17]:
df[['poverty', 'pop_determined_poverty_status']]

,poverty,pop_determined_poverty_status
1001,8422.0,54765.0
1003,21653.0,204929.0
1005,6597.0,22856.0
1007,2863.0,20468.0
1009,8220.0,57082.0
...,...,...
72145,24080.0,53158.0
72147,3600.0,8725.0
72149,10995.0,22832.0
72151,18207.0,34052.0


In [18]:
df['poverty_rate'] = df.poverty / df.pop_determined_poverty_status

df['death_rate'] = df.deaths / df.confirmed_cases

In [19]:
df.sort_values(by='poverty_rate', ascending=False).head(20)

,date,county,state_name,confirmed_cases,deaths,do_date,total_pop,households,male_pop,female_pop,...,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english,poverty_rate,death_rate
72093,2021-06-02,Maricao,Puerto Rico,150,NaN,2014-01-01,6202,1930,3069,3133,...,264,319,338,1393,344,NaN,NaN,NaN,0.641567,NaN
72055,2021-06-02,Guanica,Puerto Rico,339,NaN,2014-01-01,16783,5469,8091,8692,...,802,879,893,4091,907,NaN,NaN,NaN,0.638061,NaN
72001,2021-06-02,Adjuntas,Puerto Rico,551,NaN,2014-01-01,18181,5861,8862,9319,...,823,943,998,4406,1017,NaN,NaN,NaN,0.624828,NaN
72079,2021-06-02,Lajas,Puerto Rico,603,NaN,2014-01-01,23315,7911,11355,11960,...,991,1054,1166,4906,1096,NaN,NaN,NaN,0.614859,NaN
72073,2021-06-02,Jayuya,Puerto Rico,591,NaN,2014-01-01,14906,5087,7428,7478,...,695,905,897,3786,854,NaN,NaN,NaN,0.604096,NaN
72045,2021-06-02,Comerio,Puerto Rico,877,NaN,2014-01-01,19539,5836,9692,9847,...,647,1267,1150,4893,1279,NaN,NaN,NaN,0.602733,NaN
72019,2021-06-02,Barranquitas,Puerto Rico,1606,NaN,2014-01-01,28755,8918,14162,14593,...,1438,1460,1660,7863,2133,NaN,NaN,NaN,0.590030,NaN
72015,2021-06-02,Arroyo,Puerto Rico,454,NaN,2014-01-01,18111,6002,8514,9597,...,833,1224,1035,4411,754,NaN,NaN,NaN,0.580308,NaN
72107,2021-06-02,Orocovis,Puerto Rico,1101,NaN,2014-01-01,21407,6734,10725,10682,...,892,1414,1201,4966,1008,NaN,NaN,NaN,0.575439,NaN
72099,2021-06-02,Moca,Puerto Rico,1686,NaN,2014-01-01,36872,13278,18012,18860,...,1727,2093,1981,9353,2340,NaN,NaN,NaN,0.572168,NaN


In [20]:
df.loc[df.deaths.isna()].sort_values(by='state_name')

,date,county,state_name,confirmed_cases,deaths,do_date,total_pop,households,male_pop,female_pop,...,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english,poverty_rate,death_rate
72001,2021-06-02,Adjuntas,Puerto Rico,551,NaN,2014-01-01,18181,5861,8862,9319,...,823,943,998,4406,1017,NaN,NaN,NaN,0.624828,NaN
72109,2021-06-02,Patillas,Puerto Rico,505,NaN,2014-01-01,17334,6233,8481,8853,...,718,748,1002,4133,997,NaN,NaN,NaN,0.532710,NaN
72107,2021-06-02,Orocovis,Puerto Rico,1101,NaN,2014-01-01,21407,6734,10725,10682,...,892,1414,1201,4966,1008,NaN,NaN,NaN,0.575439,NaN
72105,2021-06-02,Naranjito,Puerto Rico,2041,NaN,2014-01-01,28557,8520,14039,14518,...,1126,1438,1650,7056,2060,NaN,NaN,NaN,0.458977,NaN
72103,2021-06-02,Naguabo,Puerto Rico,1207,NaN,2014-01-01,26266,8317,12332,13934,...,1434,1545,1503,6813,1409,NaN,NaN,NaN,0.495625,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72047,2021-06-02,Corozal,Puerto Rico,2072,NaN,2014-01-01,34165,10768,16565,17600,...,1752,1545,2089,8538,2127,NaN,NaN,NaN,0.562890,NaN
72045,2021-06-02,Comerio,Puerto Rico,877,NaN,2014-01-01,19539,5836,9692,9847,...,647,1267,1150,4893,1279,NaN,NaN,NaN,0.602733,NaN
72043,2021-06-02,Coamo,Puerto Rico,1165,NaN,2014-01-01,39265,13346,19056,20209,...,1851,2144,1901,9475,2219,NaN,NaN,NaN,0.469585,NaN
72055,2021-06-02,Guanica,Puerto Rico,339,NaN,2014-01-01,16783,5469,8091,8692,...,802,879,893,4091,907,NaN,NaN,NaN,0.638061,NaN


looks like all the nans in the deaths column are from puerto rico

In [21]:
df.dropna(subset=['deaths'], inplace=True)

In [22]:
df

,date,county,state_name,confirmed_cases,deaths,do_date,total_pop,households,male_pop,female_pop,...,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,speak_only_english_at_home,speak_spanish_at_home,speak_spanish_at_home_low_english,poverty_rate,death_rate
1001,2021-06-02,Autauga,Alabama,7161,111.0,2014-01-01,55200,21115,26874,28326,...,3009,2823,3245,13435,1865,NaN,NaN,NaN,0.153784,0.015501
1003,2021-06-02,Baldwin,Alabama,21674,311.0,2014-01-01,208107,78622,101188,106919,...,9861,11627,10456,45076,6674,NaN,NaN,NaN,0.105661,0.014349
1005,2021-06-02,Barbour,Alabama,2340,59.0,2014-01-01,25782,9186,13697,12085,...,1106,1280,1424,5216,704,NaN,NaN,NaN,0.288633,0.025214
1007,2021-06-02,Bibb,Alabama,2666,64.0,2014-01-01,22527,6840,12152,10375,...,922,1154,1169,4424,757,NaN,NaN,NaN,0.139877,0.024006
1009,2021-06-02,Blount,Alabama,6889,139.0,2014-01-01,57645,20600,28434,29211,...,3064,2998,3180,12220,1718,NaN,NaN,NaN,0.144003,0.020177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,2021-06-02,Sweetwater,Wyoming,4547,39.0,2014-01-01,44117,15871,22882,21235,...,2912,2550,2517,11878,2238,NaN,NaN,NaN,0.120311,0.008577
56039,2021-06-02,Teton,Wyoming,3785,9.0,2014-01-01,23059,9158,11911,11148,...,1193,1077,891,4857,666,NaN,NaN,NaN,0.070508,0.002378
56041,2021-06-02,Uinta,Wyoming,2279,13.0,2014-01-01,20609,7735,10505,10104,...,1503,1551,1236,5516,533,NaN,NaN,NaN,0.125116,0.005704
56043,2021-06-02,Washakie,Wyoming,924,26.0,2014-01-01,8129,3422,4137,3992,...,389,459,598,1739,127,NaN,NaN,NaN,0.123789,0.028139


In [23]:
df.corr()['poverty_rate'].sort_values(ascending=True).head(20)

median_income                                                 -0.742317
income_per_capita                                             -0.717998
median_rent                                                   -0.429244
owner_occupied_housing_units_lower_value_quartile             -0.428629
owner_occupied_housing_units_median_value                     -0.413624
renter_occupied_housing_units_paying_cash_median_gross_rent   -0.391032
owner_occupied_housing_units_upper_value_quartile             -0.385979
median_age                                                    -0.227647
white_male_45_54                                              -0.158822
white_male_55_64                                              -0.153763
white_pop                                                     -0.150660
income_150000_199999                                          -0.137414
income_125000_149999                                          -0.132596
male_45_64_graduate_degree                                    -0

In [24]:
income_cols = [col for col in df.columns if 'income' in col]
df.drop(columns=income_cols, inplace=True)

In [25]:
df.corr()['poverty_rate'].sort_values(ascending=False).head(20)

poverty_rate                                                   1.000000
gini_index                                                     0.561598
death_rate                                                     0.250064
amerindian_pop                                                 0.096304
mobile_homes                                                   0.065531
median_year_structure_built                                    0.026093
black_male_55_64                                               0.009203
black_pop                                                      0.007726
poverty                                                        0.006229
rent_burden_not_computed                                       0.004035
two_parents_not_in_labor_force_families_with_young_children   -0.000214
households_public_asst_or_food_stamps                         -0.002171
black_male_45_54                                              -0.006482
male_45_64_less_than_9_grade                                  -0

In [26]:
cols_to_drop = ['median_rent',
                'owner_occupied_housing_units_lower_value_quartile',
                'owner_occupied_housing_units_median_value',
                'renter_occupied_housing_units_paying_cash_median_gross_rent',
                'owner_occupied_housing_units_upper_value_quartile',
                'pop_determined_poverty_status',
                'poverty']
df.drop(columns=cols_to_drop, inplace=True)

In [27]:
df.isna().sum().sort_values(ascending=False).head(30)

pop_15_and_over                                                   3133
pop_5_years_over                                                  3133
pop_divorced                                                      3133
speak_spanish_at_home_low_english                                 3133
speak_spanish_at_home                                             3133
speak_only_english_at_home                                        3133
pop_widowed                                                       3133
pop_separated                                                     3133
pop_now_married                                                   3133
pop_never_married                                                 3133
aggregate_travel_time_to_work                                      148
rent_30_to_35_percent                                                1
population_1_year_and_over                                           1
commuters_by_public_transportation                                   1
pop_16

In [28]:
for col in df.columns:
    if df[col].isna().sum() > 100:
        df.drop(columns=col, inplace=True)

In [29]:
df.dropna(inplace=True)

In [30]:
df.select_dtypes(np.number)

,confirmed_cases,deaths,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,...,occupation_services,management_business_sci_arts_employed,sales_office_employed,in_grades_1_to_4,in_grades_5_to_8,in_grades_9_to_12,in_school,in_undergrad_college,poverty_rate,death_rate
1001,7161,111.0,55200,21115,26874,28326,37.8,1789,2021,1754,...,3845.0,9336.0,5331.0,3009,2823,3245,13435,1865,0.153784,0.015501
1003,21674,311.0,208107,78622,101188,106919,42.8,5855,5551,7544,...,16911.0,33923.0,22509.0,9861,11627,10456,45076,6674,0.105661,0.014349
1005,2340,59.0,25782,9186,13697,12085,39.9,717,671,912,...,1277.0,2261.0,1729.0,1106,1280,1424,5216,704,0.288633,0.025214
1007,2666,64.0,22527,6840,12152,10375,39.9,692,651,690,...,1516.0,1775.0,1531.0,922,1154,1169,4424,757,0.139877,0.024006
1009,6889,139.0,57645,20600,28434,29211,40.8,1813,1693,2118,...,2776.0,6385.0,4569.0,3064,2998,3180,12220,1718,0.144003,0.020177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,4547,39.0,44117,15871,22882,21235,34.6,1649,1618,1804,...,3260.0,6129.0,4457.0,2912,2550,2517,11878,2238,0.120311,0.008577
56039,3785,9.0,23059,9158,11911,11148,39.3,565,725,659,...,3604.0,5629.0,2537.0,1193,1077,891,4857,666,0.070508,0.002378
56041,2279,13.0,20609,7735,10505,10104,35.5,778,978,906,...,1822.0,2688.0,1661.0,1503,1551,1236,5516,533,0.125116,0.005704
56043,924,26.0,8129,3422,4137,3992,43.5,237,183,324,...,566.0,1241.0,681.0,389,459,598,1739,127,0.123789,0.028139


In [31]:
x_cols = ['death_rate', 'occupation_services', 'walked_to_work',
          'worked_at_home']
X = df[x_cols]
y = df['poverty_rate']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42)

ss = StandardScaler()
X_train_sc = ss.fit_transform(X_train)
X_test_sc = ss.transform(X_test)

### Linear Regression

In [33]:
lr = LinearRegression()
cv_score = cross_val_score(lr, X_train, y_train)
print(cv_score.mean(), cv_score.std())
cv_score

0.07946093362468847 0.022723800884808083


array([0.04177763, 0.07029697, 0.10442057, 0.10078089, 0.08002859])

### Support Vector Regressor

In [34]:
svr = SVR()

params = {
    'kernel': ['rbf', 'poly', 'sigmoid', 'linear'],
    'gamma': ['scale', 'auto'],
    'C': [.001, .01, .1, 1]
}

gs = GridSearchCV(svr, param_grid=params, verbose=3, n_jobs= -1)
gs.fit(X_train_sc, y_train)
print(gs.score(X_test_sc, y_test))
gs.best_params_

Fitting 5 folds for each of 32 candidates, totalling 160 fits
0.055178168737266


{'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}

### Random Forest Regressor

In [35]:
rfr = RandomForestRegressor()

params = {
    'n_estimators': [75, 100, 125],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 5],
}

gs = GridSearchCV(rfr, param_grid=params, verbose=10, n_jobs= -1)
gs.fit(X_train_sc, y_train)
print(gs.score(X_test_sc, y_test))
gs.best_params_

Fitting 5 folds for each of 54 candidates, totalling 270 fits
0.28767004389078554


{'max_depth': 7,
 'min_samples_leaf': 5,
 'min_samples_split': 2,
 'n_estimators': 100}

### AdaBoost Regressor

In [36]:
ada = AdaBoostRegressor()

params = {
    'n_estimators': [40, 50, 70, 100],
    'learning_rate': [1, .1, .01, .001],
    'loss': ['linear', 'square', 'exponential']
}

gs = GridSearchCV(ada, param_grid=params, verbose=3, n_jobs= -1)
gs.fit(X_train_sc, y_train)
print(gs.score(X_test_sc, y_test))
gs.best_params_

Fitting 5 folds for each of 48 candidates, totalling 240 fits
0.1906255295171806


{'learning_rate': 0.01, 'loss': 'exponential', 'n_estimators': 50}

### Gradient Boosting Regressor

In [37]:
gbr = GradientBoostingRegressor()

params = {
    'loss': ['ls', 'lad', 'huber'],
    'learning_rate': [.1, .01, .001],
    'n_estimators': [90, 100, 120],
    'max_depth': [2, 3, 4]
}

gs = GridSearchCV(gbr, param_grid=params, verbose=3, n_jobs= -1)
gs.fit(X_train_sc, y_train)
print(gs.score(X_test_sc, y_test))
gs.best_params_

Fitting 5 folds for each of 81 candidates, totalling 405 fits
0.2915419572153142


{'learning_rate': 0.1, 'loss': 'huber', 'max_depth': 3, 'n_estimators': 90}